# 🔧 Prétraitement des Données (Preprocessing)
## Music Recommendation System - MCRec-30M Dataset

---

### 📌 Rôle de ce Notebook

Ce notebook constitue la **deuxième étape essentielle** du projet. Son objectif est de **nettoyer, transformer et préparer** les données pour la modélisation en s'appuyant sur les insights de l'EDA. Il permet de :

- 🧹 **Nettoyer** les données (valeurs manquantes, doublons, anomalies)
- 🔄 **Transformer** les features (normalisation, encodage, feature engineering)
- ✂️ **Filtrer** les utilisateurs et chansons avec peu d'interactions (cold start)
- 📊 **Créer** de nouvelles features pertinentes pour la recommandation
- 🎯 **Diviser** les données en ensembles train/test de manière temporelle
- 💾 **Sauvegarder** les données prétraitées pour la modélisation

Cette étape garantit que les modèles de recommandation seront entraînés sur des données de **haute qualité**.

---

**📥 Entrée** : `data/raw/personalized_music_recommendation_dataset.csv`  
**📤 Sortie** : Données prétraitées dans `data/processed/`

---

2 : Importation des bibliothèques

In [2]:
# Importation des bibliothèques
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yaml
import joblib
import json
import os
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

# Configuration
pd.set_option('display.max_columns', None)
print("✅ Bibliothèques importées avec succès")

✅ Bibliothèques importées avec succès


 3 : Chargement de la configuration et des données

In [3]:
# Chargement de la configuration
with open('../config.yaml', 'r', encoding='utf-8') as f:
    config = yaml.safe_load(f)

# Chargement du dataset
data_path = r"C:\Users\ekoub\OneDrive\Bureau\run python\music_recommendation_system\data\raw\personalized_music_recommendation_dataset.csv"
print("Chargement des données...")
df = pd.read_csv(data_path)

print(f"✅ Dataset chargé: {df.shape[0]:,} lignes et {df.shape[1]} colonnes")
print(f"📊 Taille initiale: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

Chargement des données...
✅ Dataset chargé: 70,129 lignes et 49 colonnes
📊 Taille initiale: 96.22 MB


4 : Nettoyage des données - Doublons et valeurs manquantes

In [4]:
print("\n" + "="*80)
print("ÉTAPE 1 : NETTOYAGE DES DONNÉES")
print("="*80)

# Vérification des doublons
duplicates = df.duplicated().sum()
print(f"\n🔍 Doublons détectés: {duplicates:,}")
if duplicates > 0:
    df = df.drop_duplicates()
    print(f"✅ Doublons supprimés. Nouvelles dimensions: {df.shape}")

# Vérification des valeurs manquantes
missing = df.isnull().sum()
missing_cols = missing[missing > 0]

if len(missing_cols) > 0:
    print(f"\n⚠️ Valeurs manquantes détectées dans {len(missing_cols)} colonnes:")
    for col, count in missing_cols.items():
        pct = (count / len(df)) * 100
        print(f"  • {col}: {count:,} ({pct:.2f}%)")
    
    # Stratégie de traitement (à adapter selon vos données)
    # Pour les colonnes numériques: imputation par la médiane
    numeric_cols_missing = df.select_dtypes(include=[np.number]).columns
    for col in numeric_cols_missing:
        if df[col].isnull().sum() > 0:
            df[col].fillna(df[col].median(), inplace=True)
    
    # Pour les colonnes catégorielles: imputation par le mode
    categorical_cols_missing = df.select_dtypes(include=['object']).columns
    for col in categorical_cols_missing:
        if df[col].isnull().sum() > 0:
            df[col].fillna(df[col].mode()[0], inplace=True)
    
    print(f"✅ Valeurs manquantes traitées")
else:
    print("\n✅ Aucune valeur manquante détectée")

print(f"\n📊 Dataset après nettoyage: {df.shape[0]:,} lignes")


ÉTAPE 1 : NETTOYAGE DES DONNÉES

🔍 Doublons détectés: 0

✅ Aucune valeur manquante détectée

📊 Dataset après nettoyage: 70,129 lignes


5 : Conversion et création des features temporelles

In [5]:
print("\n" + "="*80)
print("ÉTAPE 2 : FEATURES TEMPORELLES")
print("="*80)

# Conversion du timestamp
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['hour'] = df['timestamp'].dt.hour
df['day_of_week_num'] = df['timestamp'].dt.dayofweek
df['month'] = df['timestamp'].dt.month
df['year'] = df['timestamp'].dt.year

print("✅ Features temporelles créées: hour, day_of_week_num, month, year")


ÉTAPE 2 : FEATURES TEMPORELLES
✅ Features temporelles créées: hour, day_of_week_num, month, year


 6 : Filtrage des utilisateurs et chansons (Cold Start)

In [6]:
print("\n" + "="*80)
print("ÉTAPE 3 : FILTRAGE (COLD START)")
print("="*80)

# Récupération des paramètres
min_user_interactions = config['preprocessing']['min_interactions_per_user']
min_song_interactions = config['preprocessing']['min_interactions_per_song']

print(f"\n🔧 Filtrage des utilisateurs avec < {min_user_interactions} interactions")
print(f"🔧 Filtrage des chansons avec < {min_song_interactions} interactions")

# Comptage avant filtrage
print(f"\nAvant filtrage:")
print(f"  • Utilisateurs: {df['user_id'].nunique():,}")
print(f"  • Chansons: {df['song_id'].nunique():,}")
print(f"  • Interactions: {len(df):,}")

# Filtrage itératif (utilisateurs puis chansons)
while True:
    initial_size = len(df)
    
    # Filtrer les utilisateurs
    user_counts = df['user_id'].value_counts()
    valid_users = user_counts[user_counts >= min_user_interactions].index
    df = df[df['user_id'].isin(valid_users)]
    
    # Filtrer les chansons
    song_counts = df['song_id'].value_counts()
    valid_songs = song_counts[song_counts >= min_song_interactions].index
    df = df[df['song_id'].isin(valid_songs)]
    
    # Si plus de changement, on arrête
    if len(df) == initial_size:
        break

print(f"\nAprès filtrage:")
print(f"  • Utilisateurs: {df['user_id'].nunique():,}")
print(f"  • Chansons: {df['song_id'].nunique():,}")
print(f"  • Interactions: {len(df):,}")
print(f"  • Réduction: {(1 - len(df)/initial_size)*100:.2f}%")


ÉTAPE 3 : FILTRAGE (COLD START)

🔧 Filtrage des utilisateurs avec < 3 interactions
🔧 Filtrage des chansons avec < 3 interactions

Avant filtrage:
  • Utilisateurs: 50
  • Chansons: 200
  • Interactions: 70,129

Après filtrage:
  • Utilisateurs: 50
  • Chansons: 200
  • Interactions: 70,129
  • Réduction: 0.00%


7 : Feature Engineering - Agrégations par utilisateur

In [7]:
print("\n" + "="*80)
print("ÉTAPE 4 : FEATURE ENGINEERING - UTILISATEURS")
print("="*80)

# Features agrégées par utilisateur
user_features = df.groupby('user_id').agg({
    'liked': 'mean',                    # Taux de like
    'skip_count': 'mean',               # Taux de skip moyen
    'finished_song': 'mean',            # Taux de completion
    'added_to_playlist': 'mean',        # Taux d'ajout playlist
    'repeat_count': 'mean',             # Nombre moyen de répétitions
    'listening_time_mins': 'mean',      # Temps d'écoute moyen
    'song_id': 'count'                  # Nombre total d'interactions
}).rename(columns={'song_id': 'user_total_interactions'})

user_features.columns = ['user_' + col for col in user_features.columns]

# Merge avec le dataset principal
df = df.merge(user_features, left_on='user_id', right_index=True, how='left')

print(f"✅ {len(user_features.columns)} features utilisateur créées")
print(f"Features: {list(user_features.columns)}")


ÉTAPE 4 : FEATURE ENGINEERING - UTILISATEURS
✅ 7 features utilisateur créées
Features: ['user_liked', 'user_skip_count', 'user_finished_song', 'user_added_to_playlist', 'user_repeat_count', 'user_listening_time_mins', 'user_user_total_interactions']


8 : Feature Engineering - Agrégations par chanson

In [8]:
print("\n" + "="*80)
print("ÉTAPE 5 : FEATURE ENGINEERING - CHANSONS")
print("="*80)

# Features agrégées par chanson
song_features = df.groupby('song_id').agg({
    'liked': 'mean',                    # Popularité (taux de like)
    'skip_count': 'mean',               # Taux de skip moyen
    'finished_song': 'mean',            # Taux de completion
    'added_to_playlist': 'sum',         # Nombre total d'ajouts playlist
    'play_count': 'sum',                # Nombre total d'écoutes
    'user_id': 'nunique'                # Nombre d'utilisateurs uniques
}).rename(columns={'user_id': 'song_unique_listeners'})

song_features.columns = ['song_' + col for col in song_features.columns]

# Merge avec le dataset principal
df = df.merge(song_features, left_on='song_id', right_index=True, how='left')

print(f"✅ {len(song_features.columns)} features chanson créées")
print(f"Features: {list(song_features.columns)}")


ÉTAPE 5 : FEATURE ENGINEERING - CHANSONS
✅ 6 features chanson créées
Features: ['song_liked', 'song_skip_count', 'song_finished_song', 'song_added_to_playlist', 'song_play_count', 'song_song_unique_listeners']


9 : Encodage des variables catégorielles

In [9]:
print("\n" + "="*80)
print("ÉTAPE 6 : ENCODAGE DES VARIABLES CATÉGORIELLES")
print("="*80)

# Variables à encoder
categorical_columns = ['gender', 'location', 'device_type', 'subscription_type', 
                       'time_of_day', 'day_of_week', 'preferred_genre', 'genre', 
                       'artist', 'emotion_tag', 'context_type', 'language', 'explicit']

# Dictionnaire pour sauvegarder les encoders
label_encoders = {}

for col in categorical_columns:
    if col in df.columns:
        le = LabelEncoder()
        df[col + '_encoded'] = le.fit_transform(df[col].astype(str))
        label_encoders[col] = le
        print(f"✅ {col}: {len(le.classes_)} catégories encodées")

print(f"\n✅ {len(label_encoders)} variables catégorielles encodées")


ÉTAPE 6 : ENCODAGE DES VARIABLES CATÉGORIELLES
✅ gender: 3 catégories encodées
✅ location: 5 catégories encodées
✅ device_type: 3 catégories encodées
✅ subscription_type: 2 catégories encodées
✅ time_of_day: 4 catégories encodées
✅ day_of_week: 2 catégories encodées
✅ preferred_genre: 5 catégories encodées
✅ genre: 5 catégories encodées
✅ artist: 3 catégories encodées
✅ emotion_tag: 4 catégories encodées
✅ context_type: 5 catégories encodées
✅ language: 4 catégories encodées
✅ explicit: 2 catégories encodées

✅ 13 variables catégorielles encodées


10 : Normalisation des features audio

In [10]:
print("\n" + "="*80)
print("ÉTAPE 7 : NORMALISATION DES FEATURES AUDIO")
print("="*80)

# Récupération des features audio depuis config
audio_features = config['audio_features']

print(f"Features audio à normaliser: {audio_features}")

# Normalisation avec StandardScaler
scaler = StandardScaler()
df[audio_features] = scaler.fit_transform(df[audio_features])

print(f"✅ {len(audio_features)} features audio normalisées")
print(f"Méthode: StandardScaler (moyenne=0, écart-type=1)")


ÉTAPE 7 : NORMALISATION DES FEATURES AUDIO
Features audio à normaliser: ['tempo', 'energy', 'danceability', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'loudness', 'speechiness']
✅ 9 features audio normalisées
Méthode: StandardScaler (moyenne=0, écart-type=1)


11 : Création de la matrice utilisateur-chanson

In [11]:
print("\n" + "="*80)
print("ÉTAPE 8 : CRÉATION DE LA MATRICE UTILISATEUR-CHANSON")
print("="*80)

# Création d'un score d'interaction implicite
# Score basé sur plusieurs signaux comportementaux
df['interaction_score'] = (
    df['liked'] * 5 +                      # Like = signal fort
    df['finished_song'] * 3 +              # Completion = signal fort
    df['added_to_playlist'] * 4 +          # Playlist = signal très fort
    df['repeat_count'] * 2 +               # Répétition = signal fort
    (1 - df['skip_count'] / df['skip_count'].max()) * 2  # Moins de skip = mieux
)

# Normalisation du score entre 0 et 5
df['interaction_score'] = (df['interaction_score'] - df['interaction_score'].min()) / \
                          (df['interaction_score'].max() - df['interaction_score'].min()) * 5

print(f"✅ Score d'interaction créé (range: 0-5)")
print(f"Statistiques du score:")
print(df['interaction_score'].describe())


ÉTAPE 8 : CRÉATION DE LA MATRICE UTILISATEUR-CHANSON
✅ Score d'interaction créé (range: 0-5)
Statistiques du score:
count    70129.000000
mean         1.398723
std          0.680821
min          0.000000
25%          0.825688
50%          1.330275
75%          1.880734
max          5.000000
Name: interaction_score, dtype: float64


12 : Split Train/Test temporel

In [18]:
print("\n" + "="*80)
print("ÉTAPE 9 : SPLIT TRAIN/TEST (par utilisateur, sans fuite temporelle)")
print("="*80)

# Params
test_size   = float(config['preprocessing']['test_size'])
split_type  = config['preprocessing'].get('split_type', 'temporal')
min_user_ev = int(config['preprocessing'].get('min_interactions_per_user', 2))

print(f"\n🔧 Type de split demandé: {split_type}")
print(f"🔧 Test size: {test_size}")
print(f"🔧 min_interactions_per_user: {min_user_ev}")

# 0) Sécurité: garder uniquement les users avec suffisamment d'interactions
#    (on applique ici une tolérance minimale de 2 pour pouvoir mettre 1 en train et 1 en test)
eligible = (
    df.groupby('user_id')['song_id']
      .transform('size')
      .ge(max(2, min_user_ev))
)
df_ = df.loc[eligible].copy()

# 1) tri chronologique à l'intérieur de chaque user
df_.sort_values(['user_id','timestamp'], inplace=True, kind="mergesort")

# 2) split par utilisateur (temporal = on prend les plus récentes pour test,
#    stratified = proportionnel au volume mais tjs en respectant la chronologie)
train_parts, test_parts = [], []

for uid, g in df_.groupby('user_id', sort=False):
    n = len(g)
    # nombre en test (au moins 1, au plus n-1)
    n_test = max(1, int(round(n * test_size)))
    if n_test >= n:
        n_test = 1
    n_train = n - n_test
    # découpe chronologique
    g_train = g.iloc[:n_train]
    g_test  = g.iloc[n_train:]

    # garde-fou: s'il n'y a rien en train (rare), on remet 1 du test dans train
    if g_train.empty and not g_test.empty:
        g_train = g_test.iloc[:1]
        g_test  = g_test.iloc[1:]
    # garde-fou: s'il n'y a rien en test, on met le dernier en test
    if g_test.empty and not g_train.empty:
        g_test  = g_train.iloc[-1:]
        g_train = g_train.iloc[:-1]

    train_parts.append(g_train)
    test_parts.append(g_test)

train_df = pd.concat(train_parts, ignore_index=True)
test_df  = pd.concat(test_parts,  ignore_index=True)

# 3) Statistiques de split
total = len(train_df) + len(test_df)
pct  = lambda x: (len(x) / total * 100.0 if total else 0.0)

print(f"\n✅ Split par utilisateur effectué:")
print(f"  • Train: {len(train_df):,} interactions ({pct(train_df):.1f}%)")
print(f"  • Test : {len(test_df):,} interactions ({pct(test_df):.1f}%)")

# 4) Couverture users/items & intersections
train_users = set(train_df['user_id'].unique())
test_users  = set(test_df['user_id'].unique())
common_users = train_users & test_users

train_songs = set(train_df['song_id'].unique())
test_songs  = set(test_df['song_id'].unique())
common_songs = train_songs & test_songs

print(f"\n👥 UTILISATEURS:")
print(f"  • Train uniquement: {len(train_users - test_users):,}")
print(f"  • Test uniquement : {len(test_users - train_users):,}")
print(f"  • Communs         : {len(common_users):,} ⭐ (important pour l'évaluation collab)")
print(f"  • Total           : {len(train_users | test_users):,}")

print(f"\n🎵 CHANSONS:")
print(f"  • Train uniquement: {len(train_songs - test_songs):,}")
print(f"  • Test uniquement : {len(test_songs - train_songs):,}")
print(f"  • Communes        : {len(common_songs):,}")
print(f"  • Total           : {len(train_songs | test_songs):,}")

# 5) Vérifications critiques
if len(common_users) == 0:
    print("\n❌ ERREUR: Aucun utilisateur commun entre train et test → l’évaluation collab (Surprise SVD) est impossible.")
elif len(common_users) < 10:
    print(f"\n⚠️ Attention: seulement {len(common_users)} utilisateurs communs. L’évaluation sera limitée.")
else:
    print(f"\n✅ OK: {len(common_users)} utilisateurs communs pour l’évaluation.")

# 6) Périodes temporelles (informatif)
if 'timestamp' in train_df.columns:
    try:
        print(f"\n⏰ PÉRIODES:")
        print(f"  • Train: {train_df['timestamp'].min()} → {train_df['timestamp'].max()}")
        print(f"  • Test : {test_df['timestamp'].min()} → {test_df['timestamp'].max()}")
    except Exception:
        pass

print("\n✅ Split terminé sans fuite temporelle. Rappel: normaliser les features après ce split (fit sur train).")



ÉTAPE 9 : SPLIT TRAIN/TEST (par utilisateur, sans fuite temporelle)

🔧 Type de split demandé: stratified
🔧 Test size: 0.2
🔧 min_interactions_per_user: 3

✅ Split par utilisateur effectué:
  • Train: 56,101 interactions (80.0%)
  • Test : 14,028 interactions (20.0%)

👥 UTILISATEURS:
  • Train uniquement: 0
  • Test uniquement : 0
  • Communs         : 50 ⭐ (important pour l'évaluation collab)
  • Total           : 50

🎵 CHANSONS:
  • Train uniquement: 0
  • Test uniquement : 0
  • Communes        : 200
  • Total           : 200

✅ OK: 50 utilisateurs communs pour l’évaluation.

⏰ PÉRIODES:
  • Train: 2021-01-01 00:00:00 → 2024-04-16 06:30:00
  • Test : 2024-02-13 03:30:00 → 2025-01-01 00:00:00

✅ Split terminé sans fuite temporelle. Rappel: normaliser les features après ce split (fit sur train).


13 : Préparation des données pour les modèles

In [19]:
print("\n" + "="*80)
print("ÉTAPE 10 : PRÉPARATION DES DONNÉES POUR LES MODÈLES")
print("="*80)

from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix, hstack
from sklearn.preprocessing import normalize
import joblib

# 1) Content-Based: audio (déjà normalisées) + TF-IDF (genre/artist/emotion_tag/language)
text_cols = config['text_features']           # ['genre','artist','emotion_tag','language']
audio_features = config['audio_features']     # déjà normalisées en Cellule 18

# Construire un corpus texte par chanson
text_per_song = (
    df.groupby('song_id')[text_cols]
      .agg(lambda s: ' '.join(s.astype(str)))
      .reset_index()
)
text_per_song['text'] = text_per_song[text_cols].agg(' '.join, axis=1)

# TF-IDF
vec = TfidfVectorizer(min_df=2, max_features=100_000, ngram_range=(1,2))
X_text = vec.fit_transform(text_per_song['text'])

# Agrégation audio par song_id (moyenne)
audio_agg = df.groupby('song_id')[audio_features].mean().reset_index()
# Aligner les id: merge pour garantir le même ordre
content_df = text_per_song[['song_id']].merge(audio_agg, on='song_id', how='left')

# Concaténer audio (dense->sparse) + texte (sparse)
X_audio = csr_matrix(content_df[audio_features].fillna(0.0).values)
X_content = hstack([X_audio, X_text], format='csr')
X_content = normalize(X_content)  # normalisation L2 pour cosinus

# Conserver un DataFrame léger pour export (métadonnées display)
songs_content = df[['song_id','title','artist']].drop_duplicates('song_id').merge(
    content_df[['song_id']], on='song_id', how='right'
).reset_index(drop=True)

# Sauvegarde des artefacts
joblib.dump(vec, '../data/processed/tfidf_vectorizer.pkl')
# On sauve la matrice de features comme npz + l'ordre des song_id
from scipy import sparse as sp
sp.save_npz('../data/processed/songs_content_features_matrix.npz', X_content)
songs_content.to_csv('../data/processed/songs_content_features.csv', index=False)

print(f"\n✅ Dataset Content-Based:")
print(f"  • Nombre de chansons: {X_content.shape[0]:,}")
print(f"  • Dim features: {X_content.shape[1]:,} (audio+TF-IDF)")



ÉTAPE 10 : PRÉPARATION DES DONNÉES POUR LES MODÈLES

✅ Dataset Content-Based:
  • Nombre de chansons: 200
  • Dim features: 131 (audio+TF-IDF)


14 : Sauvegarde des données prétraitées

In [20]:
print("\n" + "="*80)
print("ÉTAPE 11 : SAUVEGARDE DES DONNÉES PRÉTRAITÉES")
print("="*80)

# Créer le dossier processed si nécessaire
os.makedirs('../data/processed', exist_ok=True)

# 1. Sauvegarder les datasets complets
train_df.to_csv('../data/processed/train_data.csv', index=False)
test_df.to_csv('../data/processed/test_data.csv', index=False)
print("✅ train_data.csv et test_data.csv sauvegardés")

# 2. Sauvegarder les données pour Content-Based
songs_content.to_csv('../data/processed/songs_content_features.csv', index=False)
print("✅ songs_content_features.csv sauvegardé")

# 3. Sauvegarder les données pour Collaborative
collaborative_data.to_csv('../data/processed/collaborative_data.csv', index=False)
print("✅ collaborative_data.csv sauvegardé")

# 4. Sauvegarder les métadonnées
songs_metadata.to_csv('../data/processed/songs_metadata.csv', index=False)
print("✅ songs_metadata.csv sauvegardé")

# 5. Sauvegarder le scaler et les encoders
joblib.dump(scaler, '../data/processed/scaler.pkl')
joblib.dump(label_encoders, '../data/processed/label_encoders.pkl')
print("✅ scaler.pkl et label_encoders.pkl sauvegardés")

# 6. Sauvegarder un rapport de preprocessing
preprocessing_report = {
    'original_size': int(initial_size),
    'filtered_size': len(df),
    'reduction_pct': float((1 - len(df)/initial_size)*100),
    'train_size': len(train_df),
    'test_size': len(test_df),
    'n_users': int(df['user_id'].nunique()),
    'n_songs': int(df['song_id'].nunique()),
    'n_genres': int(df['genre'].nunique()),
    'n_artists': int(df['artist'].nunique()),
    'audio_features': audio_features,
    'categorical_features': list(label_encoders.keys()),
    'min_interactions_per_user': min_user_interactions,
    'min_interactions_per_song': min_song_interactions,
    'interaction_score_stats': df['interaction_score'].describe().to_dict()
}

with open('../data/processed/preprocessing_report.json', 'w') as f:
    json.dump(preprocessing_report, f, indent=4, default=str)
print("✅ preprocessing_report.json sauvegardé")


ÉTAPE 11 : SAUVEGARDE DES DONNÉES PRÉTRAITÉES
✅ train_data.csv et test_data.csv sauvegardés
✅ songs_content_features.csv sauvegardé
✅ collaborative_data.csv sauvegardé
✅ songs_metadata.csv sauvegardé
✅ scaler.pkl et label_encoders.pkl sauvegardés
✅ preprocessing_report.json sauvegardé


 15 : Résumé et visualisation finale

In [14]:
print("\n" + "="*80)
print("RÉSUMÉ DU PREPROCESSING")
print("="*80)

print(f"""
✅ PREPROCESSING TERMINÉ AVEC SUCCÈS!

📊 STATISTIQUES FINALES:
  • Dataset original: {initial_size:,} interactions
  • Dataset filtré: {len(df):,} interactions (-{(1-len(df)/initial_size)*100:.1f}%)
  • Train: {len(train_df):,} interactions
  • Test: {len(test_df):,} interactions
  
👥 UTILISATEURS ET CHANSONS:
  • Utilisateurs: {df['user_id'].nunique():,}
  • Chansons: {df['song_id'].nunique():,}
  • Artistes: {df['artist'].nunique():,}
  • Genres: {df['genre'].nunique():,}

🎯 FEATURES CRÉÉES:
  • Features audio normalisées: {len(audio_features)}
  • Variables catégorielles encodées: {len(label_encoders)}
  • Features utilisateur: {len([c for c in df.columns if c.startswith('user_')])}
  • Features chanson: {len([c for c in df.columns if c.startswith('song_')])}
  • Score d'interaction: ✅

💾 FICHIERS SAUVEGARDÉS:
  • data/processed/train_data.csv
  • data/processed/test_data.csv
  • data/processed/songs_content_features.csv
  • data/processed/collaborative_data.csv
  • data/processed/songs_metadata.csv
  • data/processed/scaler.pkl
  • data/processed/label_encoders.pkl
  • data/processed/preprocessing_report.json

➡️ Prochaine étape: Notebook 03_modeling.ipynb (Développement des modèles)
""")


RÉSUMÉ DU PREPROCESSING

✅ PREPROCESSING TERMINÉ AVEC SUCCÈS!

📊 STATISTIQUES FINALES:
  • Dataset original: 70,129 interactions
  • Dataset filtré: 70,129 interactions (-0.0%)
  • Train: 56,103 interactions
  • Test: 14,026 interactions

👥 UTILISATEURS ET CHANSONS:
  • Utilisateurs: 50
  • Chansons: 200
  • Artistes: 3
  • Genres: 5

🎯 FEATURES CRÉÉES:
  • Features audio normalisées: 9
  • Variables catégorielles encodées: 13
  • Features utilisateur: 8
  • Features chanson: 8
  • Score d'interaction: ✅

💾 FICHIERS SAUVEGARDÉS:
  • data/processed/train_data.csv
  • data/processed/test_data.csv
  • data/processed/songs_content_features.csv
  • data/processed/collaborative_data.csv
  • data/processed/songs_metadata.csv
  • data/processed/scaler.pkl
  • data/processed/label_encoders.pkl
  • data/processed/preprocessing_report.json

➡️ Prochaine étape: Notebook 03_modeling.ipynb (Développement des modèles)



 16 : Visualisation de la distribution du score d'interaction

In [15]:
# Visualisation du score d'interaction
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Histogram(
    x=df['interaction_score'],
    nbinsx=50,
    name='Distribution',
    marker_color='#1DB954'
))

fig.update_layout(
    title='Distribution du Score d\'Interaction',
    xaxis_title='Score d\'Interaction',
    yaxis_title='Fréquence',
    height=400,
    showlegend=False
)

fig.show()

print("\n✅ Preprocessing terminé! Les données sont prêtes pour la modélisation.")


✅ Preprocessing terminé! Les données sont prêtes pour la modélisation.


In [2]:
print("\n" + "="*80)
print("CRÉATION DE songs_content_features.csv COMPLET AVEC FEATURES AUDIO")
print("="*80)

import pandas as pd
import numpy as np

# ========================================
# CHARGER TRAIN + TEST
# ========================================

print("\n🔄 Chargement des données...")

train_df = pd.read_csv('../data/processed/train_data.csv')
test_df = pd.read_csv('../data/processed/test_data.csv')

print(f"   ✅ train_data.csv : {len(train_df):,} interactions")
print(f"   ✅ test_data.csv  : {len(test_df):,} interactions")

# Fusionner train + test pour avoir TOUTES les chansons
full_df = pd.concat([train_df, test_df], ignore_index=True)

print(f"\n✅ Total : {len(full_df):,} interactions")
print(f"   Colonnes disponibles : {full_df.shape[1]}")

# Vérifier le nombre de chansons uniques
n_unique_songs_train = train_df['song_id'].nunique()
n_unique_songs_test = test_df['song_id'].nunique()
n_unique_songs_total = full_df['song_id'].nunique()

print(f"\n📊 Chansons uniques :")
print(f"   Dans train uniquement : {n_unique_songs_train}")
print(f"   Dans test uniquement  : {n_unique_songs_test}")
print(f"   TOTAL (train + test)  : {n_unique_songs_total}")

# Chansons uniquement dans test
songs_only_in_test = set(test_df['song_id'].unique()) - set(train_df['song_id'].unique())
if songs_only_in_test:
    print(f"   ⚠️ {len(songs_only_in_test)} chansons UNIQUEMENT dans test !")
    print(f"      → Il est important de fusionner train + test")
else:
    print(f"   ✅ Toutes les chansons de test sont aussi dans train")

# ========================================
# DÉFINIR LES COLONNES À EXTRAIRE
# ========================================

# Colonnes de métadonnées des chansons (prendre la première valeur)
song_metadata_cols = [
    'song_id', 'title', 'artist', 'album', 'genre', 'release_year', 
    'language', 'duration_sec', 'popularity', 'explicit'
]

# Colonnes de features audio NUMÉRIQUES (on peut calculer la moyenne)
audio_features_numeric = [
    'tempo', 'key', 'time_signature', 'energy', 'danceability', 
    'acousticness', 'instrumentalness', 'liveness', 'valence', 'loudness', 'speechiness'
]

# Colonnes CATÉGORIELLES (prendre la première valeur, pas la moyenne)
audio_features_categorical = ['mode']  # "Major" ou "Minor"

# Toutes les colonnes nécessaires
all_song_cols = song_metadata_cols + audio_features_numeric + audio_features_categorical

# Vérifier quelles colonnes existent dans full_df
available_cols = [col for col in all_song_cols if col in full_df.columns]
missing_cols = [col for col in all_song_cols if col not in full_df.columns]

print(f"\n📊 Colonnes disponibles : {len(available_cols)}/{len(all_song_cols)}")

if missing_cols:
    print(f"⚠️ Colonnes manquantes : {missing_cols}")

# ========================================
# EXTRAIRE LES DONNÉES UNIQUES PAR CHANSON
# ========================================

print(f"\n🔄 Extraction des données uniques par chanson...")

# Préparer le dictionnaire d'agrégation
agg_dict = {}

# Pour les métadonnées : prendre la première valeur
metadata_cols_present = [col for col in song_metadata_cols if col in available_cols and col != 'song_id']
for col in metadata_cols_present:
    agg_dict[col] = 'first'

# Pour les features audio NUMÉRIQUES : prendre la moyenne
audio_numeric_present = [col for col in audio_features_numeric if col in available_cols]
for col in audio_numeric_present:
    agg_dict[col] = 'mean'

# Pour les features CATÉGORIELLES : prendre la première valeur
audio_categorical_present = [col for col in audio_features_categorical if col in available_cols]
for col in audio_categorical_present:
    agg_dict[col] = 'first'

print(f"   Agrégations définies pour {len(agg_dict)} colonnes")

# Grouper par song_id sur TOUTES les données (train + test)
songs_complete = full_df[available_cols].groupby('song_id').agg(agg_dict).reset_index()

print(f"✅ {len(songs_complete)} chansons uniques extraites")
print(f"   Colonnes finales : {songs_complete.shape[1]}")

# ========================================
# VÉRIFICATION DES DONNÉES
# ========================================

print(f"\n📋 Aperçu des données (5 premières chansons) :")
print(songs_complete.head(5))

print(f"\n📊 Statistiques des features audio NUMÉRIQUES :")
if audio_numeric_present:
    stats = songs_complete[audio_numeric_present].describe().T[['count', 'mean', 'min', 'max', 'std']]
    print(stats)

# Vérifier les valeurs manquantes
print(f"\n⚠️ Vérification des valeurs manquantes :")
missing = songs_complete.isnull().sum()
missing = missing[missing > 0]

if len(missing) > 0:
    print(f"   {len(missing)} colonnes avec des valeurs manquantes :")
    for col, count in missing.items():
        print(f"      {col}: {count} ({count/len(songs_complete)*100:.1f}%)")
    
    # Remplir les valeurs manquantes
    for col in audio_numeric_present:
        if songs_complete[col].isnull().any():
            songs_complete[col] = songs_complete[col].fillna(0)
            print(f"      → {col} : rempli par 0")
    
    for col in audio_categorical_present:
        if songs_complete[col].isnull().any():
            songs_complete[col] = songs_complete[col].fillna('Unknown')
            print(f"      → {col} : rempli par 'Unknown'")
else:
    print("   ✅ Aucune valeur manquante")

# ========================================
# SAUVEGARDER LE FICHIER
# ========================================

output_path = '../data/processed/songs_content_features.csv'
songs_complete.to_csv(output_path, index=False)

print(f"\n💾 Fichier sauvegardé : {output_path}")
print(f"   Nombre de chansons : {len(songs_complete)}")
print(f"   Nombre de colonnes : {songs_complete.shape[1]}")
print(f"   Taille du fichier : {songs_complete.memory_usage(deep=True).sum() / 1024:.2f} KB")

# ========================================
# VÉRIFICATION FINALE
# ========================================

print("\n" + "="*80)
print("VÉRIFICATION FINALE")
print("="*80)

# Recharger le fichier pour vérifier
df_check = pd.read_csv(output_path)

print(f"\n✅ Fichier rechargé avec succès")
print(f"   Lignes : {len(df_check)}")
print(f"   Colonnes : {df_check.shape[1]}")

print(f"\n📋 Liste des colonnes ({df_check.shape[1]}) :")
for i, col in enumerate(df_check.columns, 1):
    print(f"   {i:2d}. {col}")

# Afficher un exemple complet
print(f"\n📄 Exemple de chanson complète (ID {df_check.iloc[0]['song_id']}) :")
example_song = df_check.iloc[0]

# Grouper par catégories pour un meilleur affichage
print("\n   📌 MÉTADONNÉES :")
for col in ['song_id', 'title', 'artist', 'album', 'genre', 'release_year', 'language', 'duration_sec', 'popularity', 'explicit']:
    if col in df_check.columns:
        print(f"      {col:<20} : {example_song[col]}")

print("\n   🎵 FEATURES AUDIO :")
for col in audio_numeric_present + audio_categorical_present:
    if col in df_check.columns:
        value = example_song[col]
        if isinstance(value, float):
            print(f"      {col:<20} : {value:.3f}")
        else:
            print(f"      {col:<20} : {value}")

# Statistiques finales
print(f"\n📊 RÉSUMÉ DES FEATURES AUDIO :")
if 'tempo' in df_check.columns:
    print(f"   Tempo moyen       : {df_check['tempo'].mean():.3f}")
    print(f"   Tempo min/max     : {df_check['tempo'].min():.3f} / {df_check['tempo'].max():.3f}")
if 'energy' in df_check.columns:
    print(f"   Energy moyenne    : {df_check['energy'].mean():.3f}")
if 'danceability' in df_check.columns:
    print(f"   Danceability moy. : {df_check['danceability'].mean():.3f}")
if 'valence' in df_check.columns:
    print(f"   Valence moyenne   : {df_check['valence'].mean():.3f}")

if 'mode' in df_check.columns:
    mode_counts = df_check['mode'].value_counts()
    print(f"\n   Modes musicaux :")
    for mode, count in mode_counts.items():
        print(f"      {mode} : {count} chansons ({count/len(df_check)*100:.1f}%)")

# Vérifier la distribution par genre
if 'genre' in df_check.columns:
    genre_counts = df_check['genre'].value_counts()
    print(f"\n   Distribution des genres (top 5) :")
    for genre, count in genre_counts.head(5).items():
        print(f"      {genre} : {count} chansons ({count/len(df_check)*100:.1f}%)")

print("\n" + "="*80)
print("✅ FICHIER songs_content_features.csv CRÉÉ AVEC SUCCÈS !")
print("="*80)

print("\n📊 RÉCAPITULATIF :")
print(f"   ✅ {len(songs_complete)} chansons uniques (train + test)")
print(f"   ✅ {len(audio_numeric_present)} features audio numériques")
print(f"   ✅ {len(audio_categorical_present)} feature catégorielle (mode)")
print(f"   ✅ {len(metadata_cols_present)} métadonnées")

print("\nCe fichier contient maintenant :")
print("   ✅ Toutes les métadonnées (titre, artiste, genre, etc.)")
print("   ✅ Toutes les features audio NUMÉRIQUES (tempo, energy, danceability, etc.)")
print("   ✅ Les features CATÉGORIELLES (mode: Major/Minor)")
print("   ✅ TOUTES les chansons du dataset (train + test)")

print("\n➡️ L'application Streamlit peut maintenant afficher les détails audio correctement !")
print("➡️ Aucune chanson n'est manquante !")


CRÉATION DE songs_content_features.csv COMPLET AVEC FEATURES AUDIO

🔄 Chargement des données...
   ✅ train_data.csv : 56,101 interactions
   ✅ test_data.csv  : 14,028 interactions

✅ Total : 70,129 interactions
   Colonnes disponibles : 80

📊 Chansons uniques :
   Dans train uniquement : 200
   Dans test uniquement  : 200
   TOTAL (train + test)  : 200
   ✅ Toutes les chansons de test sont aussi dans train

📊 Colonnes disponibles : 22/22

🔄 Extraction des données uniques par chanson...
   Agrégations définies pour 21 colonnes
✅ 200 chansons uniques extraites
   Colonnes finales : 22

📋 Aperçu des données (5 premières chansons) :
   song_id  title   artist   album genre  release_year language  duration_sec  \
0    10000  SongC  ArtistA  AlbumY   Pop          2017  English           234   
1    10001  SongD  ArtistC  AlbumZ   Pop          2013  English           252   
2    10002  SongB  ArtistA  AlbumZ   Pop          2016    Hindi           178   
3    10003  SongB  ArtistB  AlbumZ   P